In [127]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from statsmodels.tsa.api import VAR
from sklearn.metrics import mean_squared_error, r2_score,mean_absolute_percentage_error

## Objectif du projet : 
#### Prédire le nombre de validations par jour pour toutes les gares de SNCF-Transilien entre le 1er janvier 2023 et le 30 juin 2023. 


#### x-train est un jeu de données avec 1 237 971 lignes et 6 colonnes. Il s’agit des données quotidiennes de validations des 448 stations du réseau SNCF-Transilien (comprenant toutes les gares du RER A et du RER B) entre le 1er janvier 2015 et le 31 décembre 2022 (soit 2922 jours).

#### et y-train c'est le nombre de validations par jour pour toutes les gares de SNCF-Transilien entre le 1er janvier 2015 et le 31 décembre 2022 (soit 2922 jours).

In [128]:
#Récupération des données
X = pd.read_csv("Data/train_f_x.csv")
y =  pd.read_csv("Data/y_train_sncf.csv")
final_test_data = pd.read_csv("Data/test_f_x_THurtzP.csv")

In [129]:
# Conversion de la colonne 'date' en datetime
X['date'] = pd.to_datetime(X['date'])
# Encodage de la colonne 'station'
X['station'] = X['station'].astype('category').cat.codes
# Préparation des données pour le modèle VAR
X = X.set_index('date')
X = X[['station','job', 'ferie', 'vacances']]
y = y['y']

In [130]:
#Création de l'ensemble de test et l'ensemble d'entrainement:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [131]:
# Ajustement du modèle VAR sur les données d'entraînement
model = VAR(X_train)
model_fitted = model.fit(ic='aic')
# Vérification de l'ordre optimal du lag choisi
print(f'Optimal Lag Order: {model_fitted.k_ar}')
print(model_fitted.summary())


/Users/daoudafomba/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/daoudafomba/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it is not monotonic and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


In [ ]:
# Nombre de jours à prévoir
n_forecast = len(test)

# Prévision avec le modèle VAR
forecast_input = train.values[-model_fitted.k_ar:]
forecast = model_fitted.forecast(y=forecast_input, steps=n_forecast)

# Conversion de la prévision en DataFrame
forecast_df = pd.DataFrame(forecast, index=test.index, columns=train.columns)

# Affichage des prévisions
print(forecast_df.head())

# Ajout des prévisions au DataFrame test pour comparaison
test['y_pred'] = forecast_df['y']

# Affichage des valeurs réelles vs prédites
print(test[['y', 'y_pred']])


In [ ]:
# Calcul de l'erreur quadratique moyenne
mse = mean_squared_error(test['y'], test['y_pred'])
print(f'Mean Squared Error: {mse}')


In [ ]:
mape = mean_absolute_percentage_error(y_test,y_pred)
mape

12.940642867999314